In [3]:
# import pandas
import pandas as pd  

# from apyori import apriori
from apyori import apriori

In [4]:
# load data 
movie_data = pd.read_csv('https://raw.githubusercontent.com/pirandello/apriori/master/movie_dataset.csv', 
                         header = None)
print(movie_data.shape)
movie_data.head()

(7501, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,The Revenant,13 Hours,Allied,Zootopia,Jigsaw,Achorman,Grinch,Fast and Furious,Ghostbusters,Wolverine,Mad Max,John Wick,La La Land,The Good Dunosaur,Ninja Turtles,The Good Dunosaur Bad Moms,2 Guns,Inside Out,Valerian,Spiderman 3
1,Beirut,Martian,Get Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Deadpool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,X-Men,Allied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ninja Turtles,Moana,Ghost in the Shell,Ralph Breaks the Internet,John Wick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* explore the dataset

* transform dataframe to list of lists (suitable format for apyori)

In [6]:
records = []
for i in range(len(movie_data)):
    records.append([str(movie_data.values[i, j]) for j in range(20)])
records[0:2]

[['The Revenant',
  '13 Hours',
  'Allied',
  'Zootopia',
  'Jigsaw',
  'Achorman',
  'Grinch',
  'Fast and Furious',
  'Ghostbusters',
  'Wolverine',
  'Mad Max',
  'John Wick',
  'La La Land',
  'The Good Dunosaur',
  'Ninja Turtles',
  'The Good Dunosaur Bad Moms',
  '2 Guns',
  'Inside Out',
  'Valerian',
  'Spiderman 3'],
 ['Beirut',
  'Martian',
  'Get Out',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan']]

In [7]:
len(records)

7501

* instantiate apriori and set the input params based on following constraints:
    * we want only movies that are purchased at least 40 times
    * the minimum confidence for the rules is 20%
    * the minumum lift is 3

In [9]:
association_rules = apriori(records, min_length=2,
    min_support=40/len(records),
    min_confidence=.2,
    min_lift=3
    )

* how many association rules did we obtained ?

In [10]:
association_results = list(association_rules)
len(association_results)

32

* print the first association rule

In [17]:
association_results[1]

RelationRecord(items=frozenset({'Star Wars', 'Green Lantern'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Star Wars'}), items_add=frozenset({'Green Lantern'}), confidence=0.3728813559322034, lift=4.700811850163794)])

* convert association rules to DataFrame 
    * use these columns: title_1, title_2, support, confidence, lift

In [25]:
results = []
for item in association_results:
    # first index of the inner list contains base item and add item.
    # A pair may contain more than 2 items because each "item" in the pair can contain more than 1 movie.
    pair = item[0]
    items = [item for item in pair]
    title1 = str(items[0])
    title2 = str(items[1])
    # support = (item[1]).str.replace('\w+=(\d*.\d*)', '\\1', regex=True)
    support = item[1]
    #third index of the list located at 0th of the third index of the inner list
    confidence = item[2][0][2]
    lift = item[2][0][3]
    rows = (title1, title2, support, confidence, lift)
    results.append(rows)
labels = ['Title 1','Title 2','Support','Confidence','Lift']
movie_suggestion = pd.DataFrame.from_records(results, columns = labels)
movie_suggestion.head()

,Title 1,Title 2,Support,Confidence,Lift
0,Green Lantern,Red Sparrow,0.005733,0.300699,3.790833
1,Star Wars,Green Lantern,0.005866,0.372881,4.700812
2,Jumanji,Kung Fu Panda,0.015998,0.323450,3.291994
3,Jumanji,Wonder Woman,0.005333,0.377358,3.840659
4,The Spy Who Dumped Me,Spiderman 3,0.007999,0.271493,4.122410
